In [ ]:
from train import train

args = {
    'batch_size': 128,
    'model': 'seresnext50',
    'freeze': False,
    'eval_all': True,
    'data_root': '../dataset/classify-leaves',
    'val_ratio': 0.1,
    'fold': 10,
    'lr': 0.01,
    'epoch': 100,
    'resume': False,
    'verbose': True,
    'ckpt_path': 'model.pth'
}
class AttributeDict(dict):
    __slots__ = () 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
args = AttributeDict(args)

train(args)

 85%|████████▍ | 110/130 [02:51<00:31,  1.58s/it]

109/130 loss:3.2732342828403818  acc:0.19673295454545456


 88%|████████▊ | 114/130 [02:57<00:24,  1.54s/it]